In [2]:
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.transform import factor_cmap
from bokeh.palettes import grey
from bokeh.layouts import row
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
cur.execute(songplays_by_level_select)
results = cur.fetchall()

level_play = pd.DataFrame(results, columns=['Plays', 'Level'])
level_source = ColumnDataSource(level_play)
levels = level_play['Level'].values

cur.execute(songplays_by_location_select)
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Plays', 'Location']).head(20)
locations = df['Location'].values
source = ColumnDataSource(df)

df.head()

,Plays,Location
0,691,"San Francisco-Oakland-Hayward, CA"
1,665,"Portland-South Portland, ME"
2,557,"Lansing-East Lansing, MI"
3,475,"Chicago-Naperville-Elgin, IL-IN-WI"
4,456,"Atlanta-Sandy Springs-Roswell, GA"


In [5]:
output_notebook()

s1 = figure(
    y_range=locations,
    title=f'Top {locations.size} locations by no. of song plays',
    x_axis_label='Song plays',
    y_axis_label='Location',
    plot_height=800
)
s1.hbar(
    y='Location',
    right='Plays',
    left=0,
    height=0.4,
    fill_color=factor_cmap(
        'Location',
        palette=grey(20),
        factors=locations
    ),
    fill_alpha=0.6,
    source=source
)

s2 = figure(
    x_range=levels, 
    plot_height=800, 
    plot_width=450, 
    title="Song plays by user level", 
    y_axis_label='Song plays',
    x_axis_label='Level'
)
s2.vbar(x='Level', top='Plays', width=0.5, source=level_source)

s2.xgrid.grid_line_color = None
s2.legend.orientation = "horizontal"
s2.legend.location = "top_center"

p = row(s1, s2)
show(p)

Loading BokehJS ...

In [6]:
month = 11
year = 2018
cur.execute(songplays_by_day_select, (month, year))
results = cur.fetchall()

df = pd.DataFrame(results, columns=['Plays', 'Date'])
dates = df['Date'].map(lambda day: str(day)).values
df.head()

,Plays,Date
0,11,1
1,155,2
2,100,3
3,144,4
4,356,5


In [7]:
p = figure(
    x_range=dates, 
    plot_height=800, 
    plot_width=800, 
    title=f'Song plays in the month of {month} for the year {year}',
    y_axis_label='Song plays',
    x_axis_label='Day'
)
p.vbar(x='Date', top='Plays', width=0.5, source=ColumnDataSource(df))


show(p) # show the results